# Getting started

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

`comparison_metrics` can be best used thorugh the `xarray` accessor. Comparison methods are available for `xarray` `DataArray` objects. We will also need other functionality from the package, so we import it, too:

In [ ]:
from comparson_metrics import cmp_xarray
import comparson_metrics as cmp

## Comparing 1D signals

The package comes with tutorial datasets:

In [ ]:
s1, s2 = cmp.tutorial.load_dataset('1d_signals')  # returns two xarray Data Array objects
s1

Plot data arrays and note, that x data location not always agree and one signal data point is missing and masked with `np.nan`, thus line is not drawn

In [ ]:
plt.figure()
s1.plot(linestyle='--', marker='o', color='k', label='s1')
s2.plot(linestyle='--', marker='o', color='r', label='s2')
plt.title('')
plt.ylabel('signal strenght [a.u.]')
plt.legend()
plt.show()

## Computing the absolute error

The absolute error or simply the difference between observation and prediction
\begin{equation}
d = s_1-s_2
\end{equation}
between both signals, although x-coordinates are different and a data point is "s2" is missing (`np.nan`), can be computed as follows:

In [ ]:
d = s1.cmp.compute_metric(s2, 'AE')
d

The logger statement shows, that the "observation" was interpolated onto the "prediction". "Observation" is **s1** and "prediction" is **s2**. Interpolation is done this way because the number of data points in observation is higher:

In [ ]:
d.cmp.interpolation_direction

The information is also stored as `xarray.DataArray` attribute:

In [ ]:
d.interpolation_direction

Visual inspection by plotting both signals and the difference:

In [ ]:
s1.plot(linestyle='--', marker='o', color='k', label='s1')
s2.plot(linestyle='--', marker='o', color='r', label='s2')
d.AE.plot(linestyle='--', marker='o', color='g', label='AE')
plt.ylabel('Signal and AE')
plt.legend()

The accessor has a plotting function, too:

In [ ]:
d.cmp._obj.data_vars

In [ ]:
_ = d.cmp.plot(figsize=(10, 4), marker='o', color='k')

## Vortex core (2d velocity field)

In [ ]:
ds1 = cmp.tutorial.load_dataset('vortex1')
ds2 = cmp.tutorial.load_dataset('vortex2')

In [ ]:
vortex_diff = ds2.u.cmp.compute_metric(ds1.magnitude, 'AE')

In [ ]:
vortex_diff.cmp.plot((14, 4), vmin=-1, vmax=1)

In [ ]:
vortex_diff.cmp.contourf((14, 4))

# Comparison Metrics for vector fields

On the `xr.Dataset` call `compute_metric`, provide the other dataset, the metric name and the vector components.

In [ ]:
cmp.metrics.metric_dict

## Local metric: Relevance Index
A local metric returns a scalar data variable with the same size/shape of common input data.<br>
The relevance index computes the cosine between the vectors. More information is stored in the attributes of the comparison value in the dataset.<br> In the example, the datasets are interpolated onto each other (depending which one has the higher resolution. The higher resolved dataset is interpolated on the coarser)

In [ ]:
cmp = ds1.cmp.compute_metric(ds2, 'RI', ('u', 'v'))

In [ ]:
cmp

In [ ]:
cmp.RI.plot()

## Global metric: MAE
A global metric returns a *single* value for a comparison. In this example, the MAE (Mean average error) is computed

In [ ]:
cmp = ds1.u.cmp.compute_metric(ds2.u, 'MAE')

In [ ]:
cmp

In [ ]:
cmp.MAE

In [ ]:
cmp

## Getting bibtex of a metric
Each metric has an attribute `bibtex` containing a list of bibtex strings of publications where the metric is mentioned. **Note, that this very likely is not a complete list!** Some trivial metrics may not have a bibtex at all. Please, feel free to add publications where the method is mentioned an is not yet registered in this package.

In [ ]:
from comparson_metrics.metrics import RI

Initialize a comparison method and call `get_bibtex()`. This will return a list of bibtex strings. **Note, that this very likely is not a complete list!**

In [ ]:
RI().get_bibtex()